In [ ]:
import httpx
import ssl

url = "https://wwwnew.kweather.co.kr/forecast/data/lifestyle/27230510.xml"

# SSL 설정: 보안 수준 낮춤
context = ssl.create_default_context()
context.set_ciphers("DEFAULT:@SECLEVEL=1")  # 보안 레벨 1로 낮춤

try:
    with httpx.Client(verify=context, timeout=10.0) as client:
        response = client.get(url)
        response.encoding = 'utf-8'
        print(response.text)
except Exception as e:
    print(f"Error: {e}")


In [ ]:
import httpx
import ssl
from bs4 import BeautifulSoup

field_list = ['48160720','27230510']

for code in field_list:
    url = f"https://wwwnew.kweather.co.kr/forecast/data/lifestyle/{code}.xml"

    # SSL 설정: 보안 수준 낮춤
    context = ssl.create_default_context()
    context.set_ciphers("DEFAULT:@SECLEVEL=1")  # 보안 레벨 1로 낮춤

    try:
        with httpx.Client(verify=context, timeout=10.0) as client:
            response = client.get(url)
            response.encoding = 'utf-8'
            #print(response.text)
            soup = BeautifulSoup(response.text, features='xml')
            print('------')
            
            weather_dict = {}
            weather_dict['areaName1'] = soup.find('areaName1').text
            
            day1 = soup.find('day')
            weather_dict['applyDate'] = day1.find('applyDate').text
            weather_dict['weather'] = day1.find('weather').text
            
            for hour1 in day1.find_all('hour'):
                print(hour1.find('time'))
                
            print(weather_dict)
            
    except Exception as e:
        print(f"Error: {e}")